# import

In [ ]:
import re
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split # удобный инструмент для разделения датасета
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')  # 40.000 строк
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')  # 10.000 строк 
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv') # 10.000 строк 

# возпользуемся внешним источником для анализа тональности английских слов:
positive_words = pd.read_csv('/kaggle/input/lists-of-emotion-words/positive_words.csv', names=['word'])
negative_words = pd.read_csv('/kaggle/input/lists-of-emotion-words/neg_words.csv', names=['word'])

In [ ]:
sample_submission.head(3)

In [ ]:
# ВАЖНО! для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

df = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

# **Обработка признаков**

In [ ]:
# Распределение пропусков равномерное, - обработаем их при работе с соответствующим признаком.
plt.figure(figsize=(16, 8))
sns.heatmap(df.isnull())

# 'ID_TA'

In [ ]:
# Видим задублированные значения по данному параметру:
df.ID_TA.nunique(), df[df['sample'] == 1].ID_TA.nunique()

In [ ]:
# Переведем признак ID_TA в числовой вид:
df.ID_TA = df.ID_TA.apply(lambda x: int(x[1:]))

# Добавим новый параметр соответствующим задвоенным ID_TA:
df['ID_TA_num'] = df.ID_TA.map(dict(df.ID_TA.value_counts() - 1))

МинМакс нормализация данного признака не привела к улучшению MAE.

In [ ]:
df.ID_TA.describe()

# 'Restaurant_id'

In [ ]:
df.Restaurant_id.value_counts().hist(bins=38)

In [ ]:
df.Restaurant_id.value_counts()

In [ ]:
# Рассмотрим подробнее самый часто-встречающийся id: id_227
df[df.Restaurant_id == 'id_227'].head(8)

Видим, что для одного и того же id_227 - одинаковый ранг для всех городов: Ranking=228 (sample=1), кроме тестовой базы (Ranking=3208, sample=0).
От сюда делаем вывод, что под id_227 - зашифрованы заведения не одной сети (как можно было бы предположить), а заведения имеющие Ranking=228 по своему городу.
Следовательно параметр Ranking - необходимо будет нормализовать для каждого города соответственно.

# 'Number of Reviews'

In [ ]:
# Посмотрим на распределение кол-ва отзывов по городам:
df.groupby('City')['Number of Reviews'].sum().sort_values(ascending=False).plot(kind='bar')

In [ ]:
df.groupby('City')['Number of Reviews'].mean().sort_values(ascending=False).plot(kind='bar')

In [ ]:
df['Number of Reviews'].plot()

In [ ]:
# Создадим словарь количество отзывов по городам:
rewiews_city_dict = dict(df.groupby('City')['Number of Reviews'].sum())

# Создадим признак отношения рейтинга в городе к количеству отзывов в городе:
df['relative_rank_reviews'] = df['Ranking'] / df.City.map(rewiews_city_dict)


# Добавим признак описывающий отсутсвие комментариев по заведению:
dict_null = {True: 1, False: 0}
df['null_review'] = df['Number of Reviews'].isna().map(dict_null)

# Проведем замену пропусков на среднее значение для соответствующего города:
df['Number of Reviews'][df['Number of Reviews'].isna()] = df.City.map(dict(df.groupby('City')['Number of Reviews'].mean()))

In [ ]:
# Проведем квадратическую нормальзацию параметра:
df['Number of Reviews'] = np.sqrt(df['Number of Reviews'])

In [ ]:
sns.jointplot(x='Number of Reviews', y='Ranking', data=df)

# 'Ranking'

In [ ]:
df[df['sample'] == 1].Ranking.max(), df[df['sample'] == 0].Ranking.max()

Максимальный Ranking в тренировочных данных - 16444, и в тестовых - 16443. Что говорит о том что он строится не по одному городу, - а видимо по всему макрорегиону (Европа).

In [ ]:
sns.boxplot(y='Ranking', x='Rating', data=df[df['sample'] == 1])

In [ ]:
# Исходя из выводов выше нормальзуем Ranking для каждого города по отдельности:
dict_max_rank_city = df.groupby(['City'])['Ranking'].max().to_dict()
df['Ranking'] = df.Ranking/df.City.map(dict_max_rank_city)

# Проверим есть ли зависимость Ranking по городам:
sns.boxplot(y='Ranking', x='City', data=df)

# 'Cuisine Style'

In [ ]:
# преобразуем "Cuisine Style" в list:
df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: x[2:-2].split("', '") if type(x) == str else [])

# создадим признак с кол-вом видов кухонь ('Cuisin_sum') и заменим пропуска на 1 (как минимум один вид кухни есть у каждого заведения):
df['cuisine_sum'] = df['Cuisine Style'].apply(lambda x: len(x) if x != [] else 1)

In [ ]:
df['cuisine_sum'].value_counts()

In [ ]:
# Сократим кол-во кухонь явно выпадающих в выброс (для лучшей нормализации):
df['cuisine_sum'][df['cuisine_sum'] > 9] = 9

# создадим словарь для заполнения пропусков по национальным кухням:
national_cuisines = {'Paris': 'French', 'Stockholm': 'Swedish', 'London': 'British', 'Berlin': 'German', 'Munich': 'German',
               'Oporto': 'Portuguese', 'Milan': 'Italian', 'Bratislava': 'Slovakia', 'Vienna': 'Austrian', 'Rome': 'Italian',
               'Barcelona': 'Spanish', 'Madrid': 'Spanish', 'Dublin': 'Irish', 'Brussels': 'Belgian', 'Zurich': 'Swiss',
               'Warsaw': 'Polish', 'Budapest': 'Hungarian', 'Copenhagen': 'Danish', 'Amsterdam': 'Dutch',
               'Lyon': 'French', 'Hamburg': 'German', 'Lisbon': 'Portuguese', 'Prague': 'Czech', 'Oslo': 'Norwegian',
               'Helsinki': 'Scandinavian', 'Edinburgh': 'Scottish', 'Geneva': 'Swiss', 'Ljubljana': 'Slovenian',
               'Athens': 'Greek', 'Luxembourg': 'European', 'Krakow': 'Polish'}

def national_cuisines_replace(row):
    '''
    Функция принимает строку из df и заменяет пропуски в параметре 'Cuisine Style' на
    национальную кухню для соответствующего города
    '''
    if len(row[2]) == 0:
        res = [national_cuisines[row[1]]]
    else:
        res = row[2]
    return res

df['Cuisine Style'] = df.apply(lambda row: national_cuisines_replace(row), axis=1)

In [ ]:
# обратим внимание, что для Братиславы ни один из ресторанов не включает национальный вид кухни 'Slovakia',
# зато часто встречается 'Slovenian', - предположим что это ошибка и произведем замену для этого города:
def Slovakia(list_cuisines):
    '''
    Функция производит замену 'Slovenian' на 'Slovakia' (применяем только для Братиславы)
    '''
    if 'Slovenian' in list_cuisines:
        list_cuisines.remove('Slovenian')
        list_cuisines.append('Slovakia')
    return list_cuisines

df['Cuisine Style'][df.City == 'Bratislava'] = df['Cuisine Style'][df.City == 'Bratislava'].apply(Slovakia)

In [ ]:
# создадим словарь для анализа популярности кухонь с подсчетом кол-ва раз присутствия в ресторанах:
dict_cuisines = {}
for cuisines in df['Cuisine Style']:
    for cuisine in cuisines:
        dict_cuisines[cuisine] = dict_cuisines.get(cuisine, 0) + 1

# создадим список из видов кухонь встречающихся в ресторанах более 100 раз:
list_top_cuisines = []
for cuisine, numbers in dict_cuisines.items():
    if numbers > 100:
        list_top_cuisines.append(cuisine)
print('Кол-во кухонь встречающихся в ресторанах более 100 раз: ', len(list_top_cuisines), sorted(list_top_cuisines))

In [ ]:
# удалим три вида кухни ухудшающих точность модели (выявил при тестировании MAE для разных комбинаций кухонь):
list_top_cuisines.remove('Vegetarian Friendly')
list_top_cuisines.remove('Mediterranean')
list_top_cuisines.remove('Cafe')
len(list_top_cuisines)

In [ ]:
def top_cuisine(list_cuis):
    '''
    Функция сокращает список видов кухонь до топ-списка (от 100 повторений) путём объединения остальных в категорию 'other'.
    '''
    if len(set(list_cuis) - set(list_top_cuisines)) > 0:
        res = set(list_top_cuisines) & set(list_cuis)
        res.add('other')
    else:
        res = set(list_top_cuisines) & set(list_cuis)
    return list(res)

df['Cuisine Style'] = df['Cuisine Style'].apply(top_cuisine)

# 'City'

In [ ]:
# Добавим несколько параметров сформированных на основании данных по городам ['City']:

# первый параметр - статус города: [1-столица, 0-провинция]:
capital = {'Paris': 1, 'Stockholm': 1, 'London': 1, 'Berlin': 1, 'Munich': 0, 'Oporto': 0, 'Milan': 0,
           'Bratislava': 1, 'Vienna': 1, 'Rome': 1, 'Barcelona': 0, 'Madrid': 0, 'Dublin': 1,
           'Brussels': 1, 'Zurich': 1, 'Warsaw': 1, 'Budapest': 1, 'Copenhagen': 1, 'Amsterdam': 1,
           'Lyon': 0, 'Hamburg': 0, 'Lisbon': 1, 'Prague': 1, 'Oslo': 1, 'Helsinki': 1, 'Edinburgh': 0,
           'Geneva': 1, 'Ljubljana': 1, 'Athens': 1, 'Luxembourg': 1, 'Krakow': 0}
df['capital'] = df['City'].map(capital)

# второй параметр - уровень дохода населения, где: [5 - очень высокий, 1 - очень низкий]:
income_person = {'Paris': 3, 'Stockholm': 4, 'London': 4, 'Berlin': 4, 'Munich': 3, 'Oporto': 2, 'Milan': 3,
                'Bratislava': 1, 'Vienna': 4, 'Rome': 3, 'Barcelona': 2, 'Madrid': 3, 'Dublin': 3,
                'Brussels': 5, 'Zurich': 5, 'Warsaw': 3, 'Budapest': 3, 'Copenhagen': 4, 'Amsterdam': 4,
                'Lyon': 2, 'Hamburg': 3, 'Lisbon': 3, 'Prague': 3, 'Oslo': 4, 'Helsinki': 3, 'Edinburgh': 3,
                'Geneva': 5, 'Ljubljana': 1, 'Athens': 2, 'Luxembourg': 5, 'Krakow': 2}
df['income_person'] = df['City'].map(income_person)

# третий параметр - нормированное кол-во населения:
population_norm = {'Paris': 0.2190, 'Stockholm': 0.0845, 'London': 1, 'Berlin': 0.3933, 'Munich': 0.14334,
               'Oporto': 0, 'Milan': 0.1332, 'Bratislava': 0.0229, 'Vienna': 0.1929, 'Rome': 0.2926,
               'Barcelona': 0.16351, 'Madrid': 0.35499, 'Dublin': 0.1072, 'Brussels': 0.11172, 'Zurich': 0.02190,
               'Warsaw': 0.17834, 'Budapest': 0.17337, 'Copenhagen': 0.06377, 'Amsterdam': 0.07280,
               'Lyon': 0.03192, 'Hamburg': 0.18426, 'Lisbon': 0.03084, 'Prague': 0.12579, 'Oslo': 0.05265,
               'Helsinki': 0.04798, 'Edinburgh': 0.03089, 'Geneva': 0.02967, 'Ljubljana': 0.03106,
               'Athens': 0.04793, 'Luxembourg': 0.0445, 'Krakow': 0.0620}
df['population_norm'] = df['City'].map(population_norm)

# четвертый параметр - среднегодовая температура:
aver_temperature = {'Paris': 12.3, 'Stockholm': 6.6, 'London': 10.3, 'Berlin': 10.3, 'Munich': 10,
               'Oporto': 17, 'Milan': 15, 'Bratislava': 10.5, 'Vienna': 10.4, 'Rome': 15.2,
               'Barcelona': 18.2, 'Madrid': 18, 'Dublin': 9.8, 'Brussels': 10.5, 'Zurich': 9.3,
               'Warsaw': 8.5, 'Budapest': 11.3, 'Copenhagen': 9.1, 'Amsterdam': 10.2,
               'Lyon': 12, 'Hamburg': 10, 'Lisbon': 17.5, 'Prague': 8.4, 'Oslo': 4.3,
               'Helsinki': 5.9, 'Edinburgh': 9, 'Geneva': 9, 'Ljubljana': 10.5,
               'Athens': 15, 'Luxembourg': 9, 'Krakow': 8.5}
df['aver_temperature'] = df['City'].map(aver_temperature)

# пятый параметр - кол-во веганских ресторанов на 1млн жителей:
vegan_koef = {'Paris': 4.9, 'Stockholm': 8.7, 'London': 17, 'Berlin': 12.5, 'Munich': 12.5,
               'Oporto': 10.5, 'Milan': 9.4, 'Bratislava': 4.8, 'Vienna': 16.9, 'Rome': 9.4,
               'Barcelona': 10.1, 'Madrid': 10.1, 'Dublin': 23.3, 'Brussels': 7.4, 'Zurich': 15.5,
               'Warsaw': 3.6, 'Budapest': 6.4, 'Copenhagen': 8.9, 'Amsterdam': 13.9,
               'Lyon': 4.9, 'Hamburg': 12.5, 'Lisbon': 10.5, 'Prague': 11.1, 'Oslo': 9.7,
               'Helsinki': 12.5, 'Edinburgh': 17, 'Geneva': 15.5, 'Ljubljana': 18,
               'Athens': 3.6, 'Luxembourg': 31.3, 'Krakow': 3.6}
df['vegan_koef'] = df['City'].map(vegan_koef)

# 'Reviews'

In [ ]:
def str_to_list(Review):
    '''
    Данная функция преобразует текстовый параметра Review, -
    в список из: 1) списка с двумя комментариями;
                 2) списка с двумя датами.
    '''
    if Review == '[[], []]':
        new_list = [['', ''], ['', '']]
    else:
        list_1 = Review[3:-3].split('], [\'')
        if '\', ' not in list_1[0]:
            new_list = [[list_1[0][:-1], ''], [list_1[1][-10:], '']]
        else:
            list_1_0 = list_1[0].split('\', ')
            list_1_1 = list_1[1].split('\', \'')
            new_list = [[list_1_0[0], list_1_0[1][1:-1]], [list_1_1[0][-10:], list_1_1[1][-10:]]]
    return new_list

# Преобразуем параметр Review в список:
df.Reviews.fillna('[[], []]', inplace=True)
df.Reviews = df.Reviews.apply(str_to_list)

# Выделим даты двух комментариев в отдельные парметры и преобразуем их в формат 'datetime':
df['date_1'] = df.Reviews.apply(lambda x: x[1][0])
df['date_2'] = df.Reviews.apply(lambda x: x[1][1])
df.date_1 = df.date_1.apply(pd.to_datetime)
df.date_2 = df.date_2.apply(pd.to_datetime)

# Зафиксируем дату самого свежего комметраия:
last_date = max(df.date_1.max(), df.date_2.max())
last_date

In [ ]:
# Вычислим как давно был оставлен первый комментарий (в днях, относительно последнего комментария) и переведем в числовой формат:
df['Review_1_days'] = last_date - df['date_1']
df['Review_1_days'] = df.Review_1_days.apply(lambda x: str(x)[:-14])
df['Review_1_days'] = df.Review_1_days.apply(lambda x: int(x) if x != '' else np.nan)

# Вычислим как давно был оставлен второй комментарий (в днях, относительно последнего комментария) и переведем в числовой формат:
df['Review_2_days'] = last_date - df['date_2']
df['Review_2_days'] = df.Review_2_days.apply(lambda x: str(x)[:-14])
df['Review_2_days'] = df.Review_2_days.apply(lambda x: int(x) if x != '' else np.nan)

# Добавим новый параметр - разница в днях между двумя последними комментариями:
df['review_delta'] = abs(df['Review_2_days'] - df['Review_1_days'])
# Пропуски заполним средними значением:
df.review_delta.fillna(df['review_delta'].mean(), inplace=True)

# Добавим новый параметр - как давно (в днях) был оставлен последний комментарий:
df['review_days'] = df[['Review_1_days', 'Review_2_days']].min(axis=1)
# Пропуски заполним средними значением:
df.review_days.fillna(df['review_days'].mean(), inplace=True)

df.drop(['date_1'], axis=1, inplace=True)
df.drop(['date_2'], axis=1, inplace=True)
df.drop(['Review_1_days'], axis=1, inplace=True)
df.drop(['Review_2_days'], axis=1, inplace=True)

In [ ]:
# Выделим два отдельных параметра:
df['Review_1'] = df.Reviews.apply(lambda x: x[0][0])
df['Review_2'] = df.Reviews.apply(lambda x: x[0][1])
df.drop(['Reviews'], axis=1, inplace=True)

In [ ]:
# Посмотрим на подгруженные словари позитивных и негативных слов:
positive_words.shape, negative_words.shape

In [ ]:
# добавим всем позитивным словам условный коэфициент '+1':
positive_words['koef'] = 1
# добавим всем негативным словам условный коэфициент '-1':
negative_words['koef'] = -1
# объеденим два получившихся датасэта в один:
words_emotion = pd.concat([positive_words, negative_words], ignore_index=True)

# Создадим словарь с цифровым описанием эмоционального содержания слов:
dict_emotion_words = dict(words_emotion.values)
words_emotion.sample(10)

In [ ]:
def split_words(review):
    '''
    Функция разделяет предложение на список слов (с пониженным регистром и исключая все символы)
    '''
    return re.split('[^a-z]', review.lower())

# Преобразуем оба комментария в списки слов:
df['Review_1'] = df['Review_1'].apply(split_words)
df['Review_2'] = df['Review_2'].apply(split_words)

In [ ]:
# Попытка использовать срок давности комментария - не привела к улучшению MAE
# df['Review_1_days'].fillna(1, inplace=True)
# df['Review_1_days'][df['Review_1_days'] < 100] = 1
# df['Review_1_days'][df['Review_1_days'].between(100, 365)] = 2
# df['Review_1_days'][df['Review_1_days'].between(366, 1000)] = 4
# df['Review_1_days'][df['Review_1_days'] > 1000] = 10
# df['Review_1_days'].plot(kind='hist')

# df['Review_2_days'].fillna(1, inplace=True)
# df['Review_2_days'][df['Review_2_days'] == 0] = 1
# df['Review_2_days'][df['Review_2_days'] < 100] = 1
# df['Review_2_days'][df['Review_2_days'].between(100, 365)] = 2
# df['Review_2_days'][df['Review_2_days'].between(366, 1000)] = 3
# df['Review_2_days'][df['Review_2_days'] > 1000] = 4
# df['Review_2_days'].plot(kind='hist')

In [ ]:
def review_score(review):
    '''
    Функция преобразовывает список слов в цифровую оценку (позитивную или негативную)
    '''
    x = 0
    for word in review:
        x += dict_emotion_words.get(word, 0)
    return x

# Преобразуем первый комментарий в числовую оценку тональности и обрежим выбросы:
df['Review_1'] = df['Review_1'].map(review_score)
df['Review_1'][df['Review_1'] < -3] = -3
df['Review_1'][df['Review_1'] > 4] = 4

# Попытка использовать второй комментарий - не привела к улучшению MAE
df.drop(['Review_2'], axis=1, inplace=True)

df.Review_1.value_counts()

In [ ]:
df.Review_1.plot(kind='hist', bins=7)

# 'Price Range'  -->  'Price'

In [ ]:
# переименуем и проведем числовое представление для ординального признака:
df.rename({'Price Range': 'Price'}, axis=1, inplace=True)
price = {'$': 1, '$$ - $$$': 2, '$$$$': 3}
df['Price'] = df['Price'].map(price)

# создадим параметр описывающий пропуски в изначальных данных:
df['Price_nan'] = df['Price'].isna().astype('uint8')

# df['Price'][df['Price'].isna()] = 0
df.Price.value_counts()

In [ ]:
# Проверим есть ли зависимость ценового ранга заведения по городам:
df.groupby(['City'])['Price'].median()

In [ ]:
# Создадим МОДЕЛЬ прогнозирующую пропущенные значения для Price.
# Разделение проведем по признаку 'Price_nan'.
train_price = df.query('Price_nan == 0').drop(['Price_nan'], axis=1)
test_price = df.query('Price_nan == 1').drop(['Price_nan'], axis=1)

y = train_price.Price.values            # наш таргет
X = train_price.drop(['Price', 'Rating', 'City', 'sample', 'Restaurant_id', 'Cuisine Style', 'URL_TA'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=RANDOM_SEED)

print('train_price.shape: ', train_price.shape)
print('---------')
print('X.shape: ', X.shape)
print('X_train.shape: ', X_train.shape)
print('X_test.shape: ', X_test.shape)
print('---------')
print('y.shape: ', y.shape)
print('y_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)
print('---------')
print('test_price.shape: ', test_price.shape)

In [ ]:
model_price = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)
model_price.fit(X_train, y_train)
y_pred = model_price.predict(X_test)

print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
Z = test_price.drop(['Price', 'Rating', 'City', 'sample', 'Restaurant_id', 'Cuisine Style', 'URL_TA'], axis=1)  
y_res = model_price.predict(Z)
df.Price[df['Price'].isna()] = y_res

# Матрица корреляции

In [ ]:
plt.rcParams['figure.figsize'] = (16,12)
sns.heatmap(df[df['sample'] == 1].drop(['sample'], axis=1).corr(), cmap='coolwarm', annot=True)

Попытка исключить параметры с наименьшим влиянием на целевую перенменную (корреляция около нуля) и максимальной корреляцией с другими параметрами ('income_person', 'vegan_koef', 'ID_TA_num') - не привила к улучшению MAE.

In [ ]:
# Проверим ранг получившейся матрицы:
np.linalg.matrix_rank(df.drop(['Restaurant_id', 'URL_TA', 'City', 'Cuisine Style'], axis=1))

In [ ]:
df.info()

# Dummy-параметры

In [ ]:
# добавим dummy-параметры по городам:
df = pd.get_dummies(df, columns=['City'], dummy_na=True)
# df.drop(['City'], axis=1, inplace=True)

# преобразуем обработанный параметр 'Cuisine Style' в Dummy-параметры:
df = pd.concat([df, pd.get_dummies(df['Cuisine Style'].apply(pd.Series).stack()).groupby(level=0).sum()], axis=1)
df.drop(['Cuisine Style'], axis=1, inplace=True)
df.drop(['City_nan'], axis=1, inplace=True)

# Model 

In [ ]:
# Теперь выделим тестовую часть
train_data = df.query('sample == 1').drop(['sample'], axis=1)
test_data = df.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating', 'Restaurant_id', 'URL_TA'], axis=1)

# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

print('train_data.shape: ', train_data.shape)
print('---------')
print('X.shape: ', X.shape)
print('X_train.shape: ', X_train.shape)
print('X_test.shape: ', X_test.shape)
print('---------')
print('y.shape: ', y.shape)
print('y_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)
print('---------')
print('test_data.shape: ', test_data.shape)

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
# Воспользуемся чей-то гениальной мыслью и округлим предсказанное значение до 0,5!
y_pred = np.round(model.predict(X_test)*2) / 2

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(5)

In [ ]:
test_data = test_data.drop(['Rating', 'Restaurant_id', 'URL_TA'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = np.round(model.predict(test_data)*2) / 2

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
